# Lab | Random Forests

In [172]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.model_selection import cross_val_score

In [173]:
pd.set_option('display.max_columns',None)
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')
target = pd.read_csv('target.csv')

In [174]:
X = pd.concat([categorical, numerical], axis=1)
y = target['TARGET_B']

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)


In [176]:
X_train_num = X_train.select_dtypes(np.number)
X_train_cat = X_train.select_dtypes(object)
X_test_num = X_test.select_dtypes(np.number)
X_test_cat = X_test.select_dtypes(object)

In [177]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

In [178]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='first')
X_train_cat_encoded = enc.fit_transform(X_train_cat).toarray()
X_test_cat_encoded = enc.transform(X_test_cat).toarray()
column_names = enc.get_feature_names_out(X_train_cat.columns)

In [179]:
X_train_scaled = pd.concat([pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns), pd.DataFrame(X_train_cat_encoded, columns = column_names)], axis=1)
X_test_scaled = pd.concat([pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns), pd.DataFrame(X_test_cat_encoded, columns = column_names)], axis=1)

Check Imbalanced Dataset

In [180]:
y_train = y_train.reset_index().drop(['index'], axis=1)

In [181]:
trainset= pd.concat([X_train_scaled, y_train], axis=1)
trainset

CLUSTER  DATASRCE  DOMAIN_B  ODATEW_YR  ODATEW_MM    DOB_YR    DOB_MM  \
0      0.442308       1.0  0.000000   0.214286        0.0  0.237113  0.000000   
1      1.000000       0.5  0.666667   0.928571        0.0  0.463918  0.000000   
2      0.615385       1.0  0.666667   0.357143        0.0  0.381443  0.727273   
3      0.076923       1.0  0.333333   0.428571        0.0  0.216495  0.000000   
4      0.500000       1.0  0.333333   0.785714        0.0  0.443299  0.000000   
...         ...       ...       ...        ...        ...       ...       ...   
76324  0.211538       1.0  0.000000   0.214286        0.0  0.278351  0.636364   
76325  0.634615       1.0  0.000000   0.928571        0.0  0.329897  0.000000   
76326  0.865385       1.0  0.333333   0.285714        0.0  0.000000  0.090909   
76327  0.730769       1.0  0.333333   0.214286        0.0  0.216495  0.545455   
76328  0.826923       0.5  0.333333   0.214286        0.0  0.000000  0.090909   

       MINRDATE_YR  MINRDATE_MM  MAXRDATE_YR  MAXRDATE_MM  LASTDATE_YR  \
0         0.636364     0.090909     0.681818     1.000000          0.5   
1         0.954545     0.090909     0.954545     0.090909          0.5   
2         0.727273     0.818182     0.909091     0.909091          0.0   
3         0.590909     0.909091     0.818182     0.727273          0.0   
4         0.863636     0.000000     0.909091     0.272727          0.0   
...            ...          ...          ...          ...          ...   
76324     0.909091     1.000000     0.863636     0.818182          1.0   
76325     0.954545     0.090909     0.954545     0.090909          0.5   
76326     0.545455     0.000000     0.863636     1.000000          0.5   
76327     0.545455     0.909091     0.909091     1.000000          0.5   
76328     0.681818     0.090909     0.863636     0.636364          0.5   

       LASTDATE_MM  FIRSTDATE_YR  FIRSTDATE_MM     TCODE       AGE    INCOME  \
0         0.000000      0.916667      0.181818  0.000017  0.762887  0.500000   
1         0.090909      1.000000      0.090909  0.000000  0.536082  0.666667   
2         0.909091      0.916667      0.727273  0.000017  0.608247  0.666667   
3         1.000000      0.916667      0.909091  0.000017  0.783505  0.833333   
4         0.272727      0.979167      0.000000  0.000052  0.556701  0.666667   
...            ...           ...           ...       ...       ...       ...   
76324     0.090909      0.895833      1.000000  0.000000  0.711340  0.333333   
76325     0.090909      1.000000      0.090909  0.000034  0.670103  0.333333   
76326     0.000000      0.906250      0.000000  0.000017  0.624862  0.666667   
76327     0.181818      0.895833      0.727273  0.000017  0.773196  0.666667   
76328     0.818182      0.916667      0.000000  0.000000  0.624862  0.166667   

        WEALTH1       HIT  MALEMILI   MALEVET  VIETVETS  WWIIVETS  LOCALGOV  \
0      0.666667  0.008299  0.000000  0.313131  0.101010  0.686869  0.060606   
1      1.000000  0.000000  0.000000  0.292929  0.242424  0.383838  0.070707   
2      0.111111  0.020747  0.000000  0.424242  0.161616  0.626263  0.101010   
3      0.666667  0.037344  0.010101  0.404040  0.232323  0.414141  0.080808   
4      0.222222  0.087137  0.333333  0.272727  0.292929  0.181818  0.121212   
...         ...       ...       ...       ...       ...       ...       ...   
76324  1.000000  0.020747  0.020202  0.373737  0.101010  0.323232  0.262626   
76325  0.666667  0.000000  0.000000  0.363636  0.111111  0.626263  0.040404   
76326  1.000000  0.000000  0.000000  0.353535  0.292929  0.424242  0.171717   
76327  0.333333  0.004149  0.000000  0.595960  0.222222  0.313131  0.030303   
76328  1.000000  0.000000  0.000000  0.323232  0.262626  0.373737  0.030303   

       STATEGOV    FEDGOV   WEALTH2    POP901    POP902    POP903   POP90C1  \
0      0.121212  0.034483  1.000000  0.007153  0.009129  0.007683  1.000000   
1      0.080808  0.045977  1.000000  0.045856  0.055403  0.044968  0.000000   

In [182]:
from sklearn.utils import resample

category_0 = trainset[trainset['TARGET_B'] == 0]
category_1 = trainset[trainset['TARGET_B'] == 1]

#### Apply the Random Forests algorithm but this time only by upscaling the data to deal with the imbalance

In [183]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

In [184]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)

In [185]:
X_train_upsampled = data_upsampled.drop(['TARGET_B'], axis =1)
y_train_upsampled = data_upsampled['TARGET_B'] 

In [186]:
def model_eval(model,X_train,y_train,X_test,y_test):
    # Fitting
    model.fit(X_train, y_train)
    # Predicting
    predictions = model.predict(X_test)
    # Calculating confusion matrix
    cm =confusion_matrix (y_test, predictions)
    # Printing different evaluation metrics
    print ( str(model.base_estimator_)[:-2] + " score: ", model.score(X_test, y_test))
    print ( str(model.base_estimator_)[:-2] + " precision: ", precision_score(y_test, predictions))
    print ( str(model.base_estimator_)[:-2] + " recall: ", recall_score(y_test, predictions))
    print ( str(model.base_estimator_)[:-2] + " score: ", f1_score(y_test, predictions))
    print ('\n')
    print (cm)
    return model

In [187]:
# I will use this one, because is giving me the best results

# Fit the Random Forests algorithm on the training data
rfc = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 0)
rfc.fit(X_train_upsampled, y_train_upsampled)
print('Train score: ', rfc.score(X_train_upsampled,y_train_upsampled))
print('Test score: ' , rfc.score(X_test_scaled,y_test))


Train score:  0.6208440250531138
Test score:  0.5990148299533616


In [188]:
# Make predictions on the test data
y_pred = rfc.predict(X_test_scaled)

print('accuracy:', accuracy_score(y_test, y_pred))
print("precision: ",precision_score(y_test,y_pred))
print("recall: ",recall_score(y_test,y_pred))
print("f1: ",f1_score(y_test,y_pred))

accuracy: 0.5990148299533616
precision:  0.06972833117723157
recall:  0.539
f1:  0.12348224513172969


In [189]:
rfc1 = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 0)

#### Use Feature Selections that you have learned in class to decide if you want to use all of the features (Variance Threshold, RFE, PCA, etc.)

#### Variance Threshold

In [190]:
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

In [191]:
X_added_constant = sm.add_constant(X_train_upsampled)
X_added_constant
model = sm.OLS(y_train_upsampled,X_added_constant).fit()
model.summary()
#drop values higher that 0.8

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               TARGET_B   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     32.19
Date:                Thu, 22 Jun 2023   Prob (F-statistic):               0.00
Time:                        16:32:47   Log-Likelihood:                -99722.
No. Observations:              144972   AIC:                         2.002e+05
Df Residuals:                  144617   BIC:                         2.037e+05
Df Model:                         354                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.8031      0.212      8.510      0.000       1.388       2.218
CLUSTER          0.2130      0.043      4.989      0.000       0.129       0.297
DATASRCE        -0.0130      0.005     -2.804      0.005      -0.022      -0.004
DOMAIN_B        -0.0982      0.013     -7.307      0.000      -0.124      -0.072
ODATEW_YR        0.0206      0.030      0.676      0.499      -0.039       0.080
ODATEW_MM        0.2717      0.054      5.029      0.000       0.166       0.378
DOB_YR          -0.0134      0.007     -1.863      0.062      -0.028       0.001
DOB_MM           0.0361      0.005      7.142      0.000       0.026       0.046
MINRDATE_YR      0.0653      0.018      3.726      0.000       0.031       0.100
MINRDATE_MM     -0.0093      0.004     -2.180      0.029      -0.018      -0.001
MAXRDATE_YR      0.0297      0.020      1.497      0.134      -0.009       0.069
MAXRDATE_MM     -0.0030      0.004     -0.757      0.449      -0.011       0.005
LASTDATE_YR      0.2000      0.010     20.142      0.000       0.181       0.219
LASTDATE_MM      0.0915      0.006     14.358      0.000       0.079       0.104
FIRSTDATE_YR    -1.6070      0.235     -6.833      0.000      -2.068      -1.146
FIRSTDATE_MM    -0.0114      0.005     -2.483      0.013      -0.020      -0.002
TCODE            0.0941      0.079      1.195      0.232      -0.060       0.248
AGE             -0.0703      0.011     -6.150      0.000      -0.093      -0.048
INCOME           0.0687      0.006     12.188      0.000       0.058       0.080
WEALTH1         -0.0315      0.006     -5.497      0.000      -0.043      -0.020
HIT              0.0319      0.036      0.886      0.376      -0.039       0.103
MALEMILI        -0.1725      0.052     -3.311      0.001      -0.275      -0.070
MALEVET          0.0078      0.025      0.310      0.757      -0.042       0.057
VIETVETS        -0.0297      0.017     -1.794      0.073      -0.062       0.003
WWIIVETS        -0.0453      0.017     -2.712      0.007      -0.078      -0.013
LOCALGOV         0.0418      0.058      0.726      0.468      -0.071       0.155
STATEGOV        -0.0169      0.053     -0.319      0.749      -0.121       0.087
FEDGOV          -0.1378      0.068     -2.032      0.042      -0.271      -0.005
WEALTH2          0.0023      0.006      0.419      0.675      -0.009       0.013
POP901          -0.4544      0.204     -2.228      0.026      -0.854      -0.055
POP902          -0.3416      0.166     -2.060      0.039      -0.667      -0.017
POP903           0.7560      0.134      5.637      0.000       0.493       1.019
POP90C1          1.5001      0.298      5.039      0.000       0.917       2.084
POP90C2          1.4672      0.297      4.945      0.000       0.886       2.049
POP90C3          1.4645      0.298      4.919      0.000       0.881       2.048
POP90

In [192]:
drop_list = ['MAXRDATE_MM', 'MALEVET', 'FEDGOV','WEALTH2','HUPA4', 'HHN1', 'HUPA6', 'ADI', 'IC4', 'IC22', 'TPE11', 'OCC13','EIC4', 'EIC8', 'EIC16','EC2', 'EC5','ANC6', 'HC6', 'HC11', 'HC12','HC15']

In [193]:
# Use variance threshold to remove low-variance features
variance_threshold = VarianceThreshold(threshold=0.02)
X_variance_threshold = variance_threshold.fit(X_train_upsampled)
X_variance_threshold_transform = variance_threshold.transform(X_train_upsampled)

var_list = list(X_variance_threshold.get_support())
list(zip(X_train_upsampled.columns, var_list))
[col[0] for col in zip(X_train_upsampled.columns, var_list) if col[1] == False]

removed_columns_variance_threshold = pd.DataFrame(data=(X_train_upsampled.columns,variance_threshold.variances_,X_variance_threshold.get_support()), index=('column_name','variance','statement')).T
removed_columns_variance_threshold = removed_columns_variance_threshold.loc[(removed_columns_variance_threshold['statement'] == False),:]
drop_list_variance_threshold = list(removed_columns_variance_threshold['column_name'])

In [194]:
drop_list_1 = drop_list_variance_threshold + drop_list

In [195]:
drop_list_1 = list(set(drop_list_1))
drop_list_1

['AGEC6',
 'CHILC4',
 'RHP2',
 'EIC14',
 'RHP4',
 'AFC1',
 'EIC4',
 'AFC3',
 'MINRDATE_YR',
 'HC10',
 'MAXRAMNT',
 'OEDC4',
 'ANC4',
 'HUR1',
 'TPE12',
 'CHILC5',
 'ETHC1',
 'ANC14',
 'ODATEW_MM',
 'EIC11',
 'EC6',
 'ANC8',
 'AGEC2',
 'IC18',
 'ANC11',
 'HC14',
 'ANC2',
 'LSC2',
 'EIC9',
 'IC19',
 'ETH12',
 'TPE10',
 'TPE11',
 'OCC7',
 'EIC6',
 'TIMELAG',
 'AVGGIFT',
 'LOCALGOV',
 'HHAGE2',
 'HC3',
 'EC4',
 'ETHC2',
 'RHP1',
 'IC22',
 'MAXRDATE_MM',
 'LFC10',
 'IC9',
 'WEALTH2',
 'EIC8',
 'PEC1',
 'RHP3',
 'ANC7',
 'AGE902',
 'AGE905',
 'TPE1',
 'NUMPROM',
 'HHD4',
 'HC20',
 'STATEGOV',
 'OEDC7',
 'HC9',
 'HHD11',
 'MARR4',
 'POP901',
 'ETH7',
 'TPE5',
 'HC12',
 'HU4',
 'TPE7',
 'VC4',
 'TPE9',
 'HHAS4',
 'EIC16',
 'ETHC6',
 'AFC6',
 'ANC13',
 'HHAGE3',
 'POP903',
 'HC6',
 'SEC2',
 'HHD10',
 'OCC12',
 'VOC3',
 'MALEVET',
 'HUPA6',
 'CHILC3',
 'NGIFTALL',
 'TPE4',
 'ETH4',
 'OEDC2',
 'HC21',
 'HHP2',
 'IC5',
 'EIC15',
 'HC11',
 'HC16',
 'DW3',
 'IC14',
 'ETH10',
 'IC17',
 'AGE901',
 'ET

In [196]:
X_train_upsampled_1= X_train_upsampled.drop(columns = drop_list_1)

In [197]:
X_test_1 = X_test_scaled.drop(columns = drop_list_1)


In [198]:
model_eval(rfc1,X_train_upsampled_1,y_train_upsampled,X_test_1,y_test)

DecisionTreeClassifier score:  0.6025782109731175
DecisionTreeClassifier precision:  0.07224532224532225
DecisionTreeClassifier recall:  0.556
DecisionTreeClassifier score:  0.12787488500459984


[[10943  7140]
 [  444   556]]


C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)


RandomForestClassifier(max_depth=5, max_samples=0.2, min_samples_leaf=20,
                       min_samples_split=20, random_state=0)

#### PCA

In [199]:
# Use PCA to reduce the dimensionality of the data
pca = PCA(n_components=5)
X_pca = pca.fit(X_train_upsampled)
X_pca_transform = pca.transform(X_train_upsampled)
X_test_pca = pca.transform(X_test_scaled)

In [200]:
model_eval(rfc1,X_pca_transform,y_train_upsampled,X_test_pca,y_test)

DecisionTreeClassifier score:  0.6343866268406435
DecisionTreeClassifier precision:  0.06857225350079399
DecisionTreeClassifier recall:  0.475
DecisionTreeClassifier score:  0.11984357259997476


[[11631  6452]
 [  525   475]]


C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)


RandomForestClassifier(max_depth=5, max_samples=0.2, min_samples_leaf=20,
                       min_samples_split=20, random_state=0)

In [201]:
#I see that using different tools the accurancy sometimes gets worst 

#### Re-run the Random Forest algorithm to determine if the Feature Selection has improved the results.

In [202]:
numerical =  X.select_dtypes(np.number)
categorical = X.select_dtypes(object)

In [203]:
num_scaled = pd.DataFrame(scaler.transform(numerical), columns = numerical.columns, index = numerical.index)

In [204]:
encoded = pd.DataFrame(enc.transform(categorical).toarray(), columns = column_names, index = categorical.index)

In [205]:
X_all = pd.concat([num_scaled, encoded], axis=1)

In [206]:
rfc.score(X_all,y)

0.6050077558378401

In [207]:
X['predicted_donate']=rfc.predict(X_all)

In [208]:
full_data = pd.concat([X, target], axis=1)
full_data

STATE  CLUSTER HOMEOWNR GENDER  DATASRCE RFA_2R RFA_2A GEOCODE2  \
0         IL       36        H      F         3      L      E        C   
1         CA       14        H      M         3      L      G        A   
2         NC       43        U      M         3      L      E        C   
3         CA       44        U      F         3      L      E        C   
4         FL       16        H      F         3      L      F        A   
...      ...      ...      ...    ...       ...    ...    ...      ...   
95407  other       27        H      M         3      L      G        C   
95408     TX       24        H      M         3      L      F        A   
95409     MI       30        H      M         3      L      E        B   
95410     CA       24        H      F         2      L      F        A   
95411     NC       24        U      F         3      L      G        C   

      DOMAIN_A  DOMAIN_B  ODATEW_YR  ODATEW_MM  DOB_YR  DOB_MM  MINRDATE_YR  \
0            T         2         89          1      37      12           92   
1            S         1         94          1      52       2           93   
2            R         2         90          1       0       2           91   
3            R         2         87          1      28       1           87   
4            S         2         86          1      20       1           93   
...        ...       ...        ...        ...     ...     ...          ...   
95407        C         2         96          1       0       2           96   
95408        C         1         96          1      50       1           96   
95409        C         3         95          1      38       1           96   
95410        C         1         86          1      40       5           90   
95411        C         1         88          1      18       1           88   

       MINRDATE_MM  MAXRDATE_YR  MAXRDATE_MM  LASTDATE_YR  LASTDATE_MM  \
0                8           94            2           95           12   
1               10           95           12           95           12   
2               11           92            7           95           12   
3               11           94           11           95           12   
4               10           96            1           96            1   
...            ...          ...          ...          ...          ...   
95407            2           96            2           96            2   
95408            3           96            3           96            3   
95409            3           95            1           96           10   
95410           11           96            8           97            1   
95411           12           93           12           96           12   

       FIRSTDATE_YR  FIRSTDATE_MM  TCODE        AGE  INCOME  WEALTH1  HIT  \
0                89            11      0  60.000000       5        9    0   
1                93            10      1  46.000000       6        9   16   
2                90             1      1  61.611649       3        1    2   
3                87             2      0  70.000000       1        4    2   
4                79             3      0  78.000000       3        2   60   
...             ...           ...    ...        ...     ...      ...  ...   
95407            96             2      1  61.611649       5        9    0   
95408            96             3      1  48.000000       7        9    1   
95409            94            10      1  60.000000       5        9    0   
95410            86            12      0  58.000000       7        9    0   
95411            88             3      2  80.000000       5        8    3   

       MALEMILI  MALEVET  VIETVETS  WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  \
0             0       39        34        18        10         2       1   
1             0       15        55        11         6         2       1   
2             0       20        29        33         6         8       1   
3             0       23        14        31       

In [209]:
full_data.to_csv("full_data.csv")

Discuss the output and its impact in the business scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the business?

In [2]:
#The result is good but if we have more false positives means that we will wasted the resources on someone who is not likely to donate, and in the false negative we could lose a potential donor 